<a href="https://colab.research.google.com/github/KelseyWang2022/Evaluation-algorithm-of-bus-route/blob/main/LSTM_VEHICLE_TRAJECTORY%2BPREDICITON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('/content/gdrive/MyDrive/HCMC_vehicle__dataset.csv')
# one-hot encode the class_name feature
class_name_encoded = pd.get_dummies(data['Class_name'])
# concatenate encoded class_name to the data
data = pd.concat([data, class_name_encoded], axis=1)

# # drop the original class_name column
data = data.drop(['Class_name'], axis=1)
data


,Unnamed: 0,arrival,Track_ID,X_min,X_max,Y_min,Y_max,X_min_1,X_max_1,Y_min_1,...,Y_max_7,X_min_8,X_max_8,Y_min_8,Y_max_8,Bicycle,Bus,Car,Motorcycle,Truck
0,0,0,1,273,371,154,230,275,373,153,...,229,295,386,156,229,0,0,1,0,0
1,1,3,1,275,373,153,230,279,376,153,...,229,298,386,156,228,0,0,1,0,0
2,2,6,1,279,376,153,230,282,379,152,...,228,299,387,156,228,0,0,1,0,0
3,3,8,1,282,379,152,229,286,379,154,...,228,301,388,155,226,0,0,1,0,0
4,4,11,1,286,379,154,228,289,380,155,...,226,302,388,155,226,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178877,178877,48304,16313,313,331,147,180,314,331,148,...,188,320,343,150,190,0,0,0,1,0
178878,178878,48307,16313,314,331,148,180,315,333,148,...,190,320,343,152,191,0,0,0,1,0
178879,178879,48310,16313,315,333,148,181,316,336,147,...,191,321,345,153,193,0,0,0,1,0
178880,178880,48311,16313,316,336,147,183,317,338,147,...,193,322,348,154,195,0,0,0,1,0


In [3]:
from sklearn.model_selection import train_test_split

features = ['Bicycle', 'Bus', 'Car', 'Motorcycle', 'Truck',
            'X_min', 'Y_min', 'X_max', 'Y_max',
            'X_min_1', 'Y_min_1', 'X_max_1', 'Y_max_1',
            'X_min_2', 'Y_min_2', 'X_max_2', 'Y_max_2',
            'X_min_3', 'Y_min_3', 'X_max_3', 'Y_max_3',
            'X_min_4', 'Y_min_4', 'X_max_4', 'Y_max_4',
            'X_min_5', 'Y_min_5', 'X_max_5', 'Y_max_5',
            'X_min_6', 'Y_min_6', 'X_max_6', 'Y_max_6',
            'X_min_7', 'Y_min_7', 'X_max_7', 'Y_max_7',
            ]

target = ['X_max_8', 'X_min_8', 'Y_max_8', 'Y_min_8']


In [4]:
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)# Split the data into training and test sets
timesteps = X_train.shape[1]#the dimensions of feature
X_train = X_train.values.reshape((X_train.shape[0], timesteps, 1))#X_train.shape is  (160993, 37, 1), #the input is 3D,(batch_size, samples, features), link：https://blog.csdn.net/Fwuyi/article/details/123409946
X_test = X_test.values.reshape((X_test.shape[0], timesteps, 1))#X_test.shape is  (17889, 37, 1)
y_train = y_train.values.reshape((y_train.shape[0], 4))#y_train.shape is  (160993, 4)
y_test = y_test.values.reshape((y_test.shape[0], 4))#y_test.shape is  (17889, 4)


In [5]:
%cd /content

/content


In [13]:
from keras.layers import Input, Dense, LSTM, Conv1D, MaxPooling1D, Flatten, GRU
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping#earlystopping is used for preventing overfitting
import time

#1.define the input shape
start = time.time()
input_shape = (X_train.shape[1], X_train.shape[2])#(37,1), Define the input shape, it should be 3D (samples, timesteps, features)(batch_size, timesteps, features)


#2. Define the model
model = Sequential()
model.add(LSTM(50, input_shape=input_shape))
model.add(Dense(4)) # 2 output units for X_8, Y_8
model.compile(loss='mean_squared_error', optimizer='adam')

#3. train the model
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test), callbacks=EarlyStopping(monitor='val_loss', patience=2))#early stopping is used to solve the problem of overfitting
print('Training time: ', time.time() - start)

#4.evaluate the model
score = model.evaluate(X_test, y_test, batch_size=32)
print("MSE: %.2f" % score)

#5. save the definition and weights of the model
model.save('LSTM.h5')#save the model definition and weights,so you can load this model by using lstm.h5,the structure of Transformer also defined in this file


5032/5032 [==============================] - 105s 21ms/step - loss: 46022.7734 - val_loss: 21340.9961
Training time:  105.89327120780945
560/560 [==============================] - 3s 6ms/step - loss: 21340.9961
MSE: 21341.00


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NameError: ignored

In [19]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score#load the evaluation metrics
#6.load this model
LSTM = load_model('/content/LSTM.h5')#by using kreas, model.save is using to save the definition and weights of NN, and load_model is using to load model.

#7. predict the data
prediction = LSTM.predict(X_test)# .predict is using for prediction
prediction = pd.DataFrame(prediction)
prediction.to_csv('prediction.csv')
print(mean_absolute_error(y_test, prediction))


560/560 [==============================] - 3s 5ms/step
106.13303298370177


# New section